In [48]:
import os

SEASONS = list(range(2016,2024))

DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")


In [53]:
os.makedirs(STANDINGS_DIR, exist_ok=True)
os.makedirs(SCORES_DIR, exist_ok=True)

In [54]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [55]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [56]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [57]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/leagues/NBA_2016_games-november.html
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.c

In [58]:
standings_files = os.listdir(STANDINGS_DIR)

In [59]:
standings_files

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2023_games-may.html',
 'NBA_2019_games-april.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2016_games-april.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2017_games-february.html',
 'NBA_2016_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-november.html',
 'NBA_2023_games-december.html',
 'NBA_2017_games-april.html',
 'NBA_2022_games-november.html',
 'NBA_2016_games-january.ht

In [60]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
import pandas as pd

for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

76ers vs Hornets, April 1, 2016 | Basketball-Reference.com
Mavericks vs Pistons, April 1, 2016 | Basketball-Reference.com
Nets vs Knicks, April 1, 2016 | Basketball-Reference.com
Cavaliers vs Hawks, April 1, 2016 | Basketball-Reference.com
Raptors vs Grizzlies, April 1, 2016 | Basketball-Reference.com
Magic vs Bucks, April 1, 2016 | Basketball-Reference.com
Timberwolves vs Jazz, April 1, 2016 | Basketball-Reference.com
Heat vs Kings, April 1, 2016 | Basketball-Reference.com
Celtics vs Warriors, April 1, 2016 | Basketball-Reference.com
Wizards vs Suns, April 1, 2016 | Basketball-Reference.com
Pacers vs 76ers, April 2, 2016 | Basketball-Reference.com
Pistons vs Bulls, April 2, 2016 | Basketball-Reference.com
Raptors vs Spurs, April 2, 2016 | Basketball-Reference.com
Kings vs Nuggets, April 2, 2016 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/201604020POR.html
Heat vs Trail Blazers, April 2, 2016 | Basketball-Reference.com
Pelicans vs Nets, Ap